# 1)

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('test.db')
cursor = conn.cursor()

In [3]:
# 创建table_a
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS table_a (
        uid BIGINT PRIMARY KEY,
        install_time DATETIME,
        last_login_time DATETIME);
    """)

# 插入数据
cursor.execute(
    """
    INSERT INTO table_a 
    VALUES
        (10001, '2018-04-15 22:10:32', '2018-06-30 20:56:01'),
        (10002, '2018-05-01 14:32:08', '2018-08-05 16:43:44'),
        (10003, '2018-05-10 10:03:55', '2018-08-15 12:12:23'),
        (10004, '2018-05-20 21:32:43', '2018-08-31 04:56:00'),
        (10005, '2018-05-30 23:11:45', '2018-07-15 23:22:12');
    """
    )

# 创建table_b
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS table_b(
        uid BIGINT,
        pay_time DATETIME,
        money DOUBLE);
    """
    )

# 插入数据
cursor.execute(
    """
    INSERT INTO table_b
    VALUES
        (10001, '2018-05-10 22:10:32', 6),
        (10001, '2018-06-06 23:11:45', 10),
        (10003, '2018-08-10 14:32:08', 30),
        (10003, '2018-07-12 23:22:12', 30),
        (10004, '2018-08-10 12:12:23', 30),
        (10004, '2018-08-10 12:12:25', 30);
    """
    )

conn.commit()

In [4]:
datas = cursor.execute(
            """
            SELECT 
              COUNT(a.uid) AS 用户总人数, 
              COUNT(DISTINCT c.uid) AS 付费总人数,
              COUNT(*) AS 付费总次数,
              SUM(money) AS 付费总金额
            FROM
            (
              (SELECT * FROM table_a
              WHERE
                DATE(install_time) BETWEEN '2018-05-01' AND '2018-05-30'
                AND 
                DATE(last_login_time) >= '2018-08-05'
              ) a
              INNER JOIN
              (SELECT * FROM table_b
              WHERE
                DATE(pay_time) >= '2018-08-05'
              ) b
              ON a.uid=b.uid
            ) c
            """
        )

datas.fetchall()

[(3, 2, 3, 90.0)]

# 2)

In [5]:
# 创建 PropUse 表
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS PropUse (
        UserID BIGINT,
        Dates DATETIME);
    """)

# 插入数据
cursor.execute(
    """
    INSERT INTO PropUse
    VALUES
        (10001, '2018-06-01 22:10:32'),
        (10002, '2018-06-01 14:32:08'),
        (10003, '2018-08-01 10:03:55'),
        (10004, '2018-06-01 21:32:43'),
        (10005, '2018-06-02 23:11:45');
    """
    )

# 创建Payment 表
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS Payment(
        UserID BIGINT,
        Dates DATETIME);
    """
    )

# 插入数据
cursor.execute(
    """
    INSERT INTO Payment
    VALUES
        (10001, '2018-06-01 22:10:32'),
        (10002, '2018-06-01 23:11:45'),
        (10003, '2018-08-10 14:32:08'),
        (10043, '2018-06-01 12:12:23'),
        (10071, '2018-06-02 12:12:25');
    """
    )

conn.commit()

In [8]:
datas = cursor.execute(
            """
            SELECT UserID,DATE(Dates) FROM PropUse
            WHERE 
                DATE(Dates) BETWEEN '2018-06-01' AND '2018-06-30'
            EXCEPT
            SELECT UserID,DATE(Dates) FROM Payment
            WHERE
              DATE(Dates) BETWEEN '2018-06-01' AND '2018-06-30'
            """
        )
datas.fetchall()

[(10004, '2018-06-01'), (10005, '2018-06-02')]

In [9]:
cursor.close()
conn.close()